## Generate data

In [ ]:
import lsdb
import pandas as pd
import numpy as np

from astropy.coordinates import SkyCoord

#### Delve

In [ ]:
full_delve = lsdb.read_hats(
    "https://data.lsdb.io/hats/delve_dr2",
    columns=[
        "QUICK_OBJECT_ID",
        "RA",
        "DEC",
        "MAG_PSF_G",
        "MAG_PSF_R",
        "MAG_PSF_I",
        "MAG_PSF_Z",
        "MAGERR_PSF_G",
        "MAGERR_PSF_R",
        "MAGERR_PSF_I",
        "MAGERR_PSF_Z",
    ],
)

In [ ]:
full_delve.plot_pixels()

In [ ]:
full_delve.all_columns

In [ ]:
cone_delve = full_delve.cone_search(ra=8.8567, dec=11.8167, radius_arcsec=25 * 60)
small_cone_delve = cone_delve.cone_search(ra=8.8567, dec=11.8167, radius_arcsec=100, fine=True)

In [ ]:
cone_delve.plot_pixels()

In [ ]:
cone_delve.to_hats("m67/delve_cone", overwrite=True)
small_cone_delve.to_hats("m67/delve_cone_small", overwrite=True)

#### PS1

In [ ]:
full_ps1 = lsdb.read_hats(
    "s3://stpubdata/panstarrs/ps1/public/hats/otmo",
    margin_cache="s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs",
    columns=[
        "objName",
        "objID",
        "surveyID",
        "qualityFlag",
        "raMean",
        "decMean",
        "raMeanErr",
        "decMeanErr",
        "pmra",
        "pmdec",
        "pmraErr",
        "pmdecErr",
        "epochMean",
        "gMeanPSFMag",
        "gMeanPSFMagErr",
        "gMeanPSFMagStd",
        "gMeanPSFMagNpt",
        "gMeanPSFMagMin",
        "gMeanPSFMagMax",
        "gFlags",
        "rMeanPSFMag",
        "rMeanPSFMagErr",
        "rMeanPSFMagStd",
        "rMeanPSFMagNpt",
        "rMeanPSFMagMin",
        "rMeanPSFMagMax",
        "rFlags",
        "iMeanPSFMag",
        "iMeanPSFMagErr",
        "iMeanPSFMagStd",
        "iMeanPSFMagNpt",
        "iMeanPSFMagMin",
        "iMeanPSFMagMax",
        "iFlags",
        "zMeanPSFMag",
        "zMeanPSFMagErr",
        "zMeanPSFMagStd",
        "zMeanPSFMagNpt",
        "zMeanPSFMagMin",
        "zMeanPSFMagMax",
        "zFlags",
        "yMeanPSFMag",
        "yMeanPSFMagErr",
        "yMeanPSFMagStd",
        "yMeanPSFMagNpt",
        "yMeanPSFMagMin",
        "yMeanPSFMagMax",
        "yFlags",
    ],
)

In [ ]:
full_ps1.plot_pixels()

In [ ]:
full_ps1.hc_structure.catalog_info

In [ ]:
cone_ps1 = full_ps1.cone_search(ra=8.8567, dec=11.8167, radius_arcsec=25 * 60, fine=True)
small_cone_ps1 = cone_ps1.cone_search(ra=8.8567, dec=11.8167, radius_arcsec=100, fine=True)

In [ ]:
cone_ps1.plot_pixels()

In [ ]:
cone_ps1.to_hats("m67/ps1_cone")
small_cone_ps1.to_hats("m67/ps1_cone_small")

### Crossmatch results

In [ ]:
small_cone_ps1 = lsdb.open_catalog("m67/ps1_cone_small")
small_cone_ps1_df = small_cone_ps1.compute()

In [ ]:
small_cone_delve = lsdb.open_catalog("m67/delve_cone_small")
small_cone_delve_df = small_cone_delve.compute()

In [ ]:
xmatch_radius = 3600

xmatches = {"id_ps1": [], "id_delve": [], "_dist_arcsec": [], "_mag_diff": []}

for index, row in small_cone_ps1_df.iterrows():
    matches = []

    # Calculate separation for each pair of objects
    coords_ps1 = SkyCoord(row["raMean"], row["decMean"], unit="deg")
    delve_ra = small_cone_delve_df["RA"].values
    delve_dec = small_cone_delve_df["DEC"].values
    coords_delve = SkyCoord(ra=delve_ra, dec=delve_dec, unit="deg")
    dists_arcsec = coords_ps1.separation(coords_delve).arcsec

    # Calculate magnitude difference for each object on the right
    mag_diffs = np.abs(small_cone_delve_df["MAG_PSF_R"].values - row["rMeanPSFMag"])

    # Keep only those within the crossmatching radius
    matches = list(zip(small_cone_delve_df["QUICK_OBJECT_ID"].values, dists_arcsec, mag_diffs))
    matches = [m for m in matches if m[1] < xmatch_radius]

    # Select the match with closest magnitude
    sorted_matches = sorted(matches, key=lambda x: (x[1], x[2]))
    best_match = sorted_matches[0] if sorted_matches else None
    if best_match is None:
        continue

    # Append results to create final DataFrame
    xmatches["id_ps1"].append(row["objID"])
    xmatches["id_delve"].append(best_match[0])
    xmatches["_dist_arcsec"].append(best_match[1])
    xmatches["_mag_diff"].append(best_match[2])

pd.DataFrame.from_dict(xmatches).to_csv("expected_results/xmatch_mags_rband.csv")